ARQUTECTURA REST

In [1]:
from flask import Flask, jsonify, request
import json
import pyodbc
import pandas as pd
from datetime import date
from flask_httpauth import HTTPBasicAuth
    
app = Flask(__name__)

auth = HTTPBasicAuth()

@auth.verify_password
def verify_password(username, password):
    with open('users.pass', 'r') as file:
            data = file.readlines()
            users=dict([i.split(':') for i in data])
    if username in users and password==users[username][:-1]: 
        # remove las character \n
        return True
    return False

server = '127.0.0.1,6789'
database = 'ssdd'
username = 'sa'
password = 'icaiSQL2019'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server}; \
                      SERVER='+server+';DATABASE='+database+'; \
                      UID='+username+';PWD='+password)
cursor=cnxn.cursor()

# Pass the required route to the decorator.    
@app.route("/")
def index():
    return "BIENVENIDO AL SERVIVICIO REST PARA CONSULTAR INFORMACIÓN SOBRE EL SERVICIO DE BICICLETAS DE ALQUILER EN MADRID"

@app.route('/api/v1.0/data/cantidad-fechas',  methods=['GET'])
def getCantidadFechas():
    # http://127.0.0.1:6789/api/v1.0/data/cantidad-fechas
    query = "SELECT DISTINCT fecha FROM bicimad"
    df = cursor.execute(query)
    
    rows = df.fetchall()
    data_frame = pd.DataFrame([list(row) for row in rows],columns=["fecha"])
    return data_frame.to_html()

@app.route('/api/v1.0/data/estaciones-origen',  methods=['GET'])
def getStartStations():
    # http://127.0.0.1:6789/api/v1.0/data/estaciones-origen
    query = "SELECT DISTINCT idunplug_base FROM bicimad"  
    df = cursor.execute(query)
    
    rows = df.fetchall()
    data_frame = pd.DataFrame([list(row) for row in rows],columns=["idunplug_base"])
    return data_frame.to_html()

@app.route('/api/v1.0/data/estaciones-destino',  methods=['GET'])
def getFinalStations():
    # http://127.0.0.1:6789/api/v1.0/data/estaciones-destino
    query = "SELECT DISTINCT idplug_base FROM bicimad"
    df = cursor.execute(query)
    
    rows = df.fetchall()
    data_frame = pd.DataFrame([list(row) for row in rows],columns=["idplug_base"])
    return data_frame.to_html()

@app.route('/api/v1.0/data/movimientos-dia/<fecha>',  methods=['GET'])
def getDayMovement(fecha):
    # copiar y pegar esta url para comprobar --> si funciona
    # http://127.0.0.1:6789/api/v1.0/data/movimientos-dia/2021-07-05
    df = cursor.execute("SELECT * FROM bicimad WHERE fecha = ?", str(fecha))
    rows = df.fetchall()
    data_frame = pd.DataFrame([list(row) for row in rows],columns=["fecha", "ageRange", "user_type", "idunplug_station", "idplug_station", "idunplug_base", "idplug_base", "travel_time", "Fichero"])
    return data_frame.to_html()

@app.route('/api/v1.0/data/movimientos-origen/<fecha>&<idunplug_station>',  methods=['GET'])
def getStations(fecha,idunplug_station):
    # http://127.0.0.1:6789/api/v1.0/data/movimientos-origen/2019-01-06&90 
    df = cursor.execute("SELECT * FROM bicimad WHERE fecha = ? AND idunplug_station = ?",str(fecha),str(idunplug_station))
    rows = df.fetchall()
    data_frame = pd.DataFrame([list(row) for row in rows],columns=["fecha", "ageRange", "user_type", "idunplug_station", "idplug_station", "idunplug_base", "idplug_base", "travel_time", "Fichero"])
    return data_frame.to_html()

@app.route('/api/v1.0/data/movimientos-inicio/<fecha>&<idplug_station>',  methods=['GET'])
def getMovementStart(fecha,idplug_station):
    # http://127.0.0.1:6789/api/v1.0/data/movimientos-destino/2019-01-06&90 
    df = cursor.execute("SELECT * FROM bicimad WHERE fecha = ? AND idplug_station = ?",str(fecha),str(idplug_station))
    rows = df.fetchall()
    data_frame = pd.DataFrame([list(row) for row in rows],columns=["fecha", "ageRange", "user_type", "idunplug_station", "idplug_station", "idunplug_base", "idplug_base", "travel_time", "Fichero"])
    return data_frame.to_html()

@app.route('/api/v1.0/data/movimientos-destino/<fecha>&<idplug_station>',  methods=['GET'])
def getMovementEnd(fecha,idplug_station):
    # http://127.0.0.1:6789/api/v1.0/data/movimientos-destino/2019-01-06&90 
    df = cursor.execute("SELECT * FROM bicimad WHERE fecha = ? AND idplug_station = ?",str(fecha),str(idplug_station))
    rows = df.fetchall()
    data_frame = pd.DataFrame([list(row) for row in rows],columns=["fecha", "ageRange", "user_type", "idunplug_station", "idplug_station", "idunplug_base", "idplug_base", "travel_time", "Fichero"])
    return data_frame.to_html()

@app.route('/api/v1.0/data/movimientos-inicio-destino/<fecha>&<idunplug_station>&<idplug_station>',  methods=['GET'])
def getMovementStartEnd(fecha,idunplug_station,idplug_station):
    # http://127.0.0.1:6789/api/v1.0/data/movimientos-inicio-destino/2019-01-06&90&83
    df = cursor.execute("SELECT * FROM bicimad WHERE fecha = ? AND idunplug_station = ? AND idplug_station = ?",str(fecha),str(idunplug_station),str(idplug_station))
    rows = df.fetchall()
    data_frame = pd.DataFrame([list(row) for row in rows],columns=["fecha", "ageRange", "user_type", "idunplug_station", "idplug_station", "idunplug_base", "idplug_base", "travel_time", "Fichero"])
    return data_frame.to_html()

@app.route('/api/v1.0/data/movimientos-inicio-destino-tiempo/<fecha>&<idunplug_station>&<idplug_station>&<travel_time>',  methods=['GET'])
def getMovementStartEndTime(fecha,idunplug_station,idplug_station,travel_time):
    # http://127.0.0.1:6789/api/v1.0/data/movimientos-inicio-destino-tiempo/2019-01-06&90&83&400
    df = cursor.execute("SELECT * FROM bicimad WHERE fecha = ? AND idunplug_station = ? AND idplug_station = ? AND travel_time < ?",str(fecha),str(idunplug_station),str(idplug_station),str(travel_time))
    rows = df.fetchall()
    data_frame = pd.DataFrame([list(row) for row in rows],columns=["fecha", "ageRange", "user_type", "idunplug_station", "idplug_station", "idunplug_base", "idplug_base", "travel_time", "Fichero"])
    return data_frame.to_html()

@app.route('/api/v1.0/data/nuevo',  methods=['POST'])
@auth.login_required
def putNewRow():
    # http://127.0.0.1:6789/api/v1.0/data/nuevo
    
    #Recogemos los valores introducidos por postman en el body
    datos = request.get_json()
    #Vamos almacenando las variables para insertarlas en la query
    fecha = datos['fecha']
    ageRange = datos['ageRange']
    user_type = datos['user_type']
    idunplug_station = datos['idunplug_station']
    idplug_station = datos['idplug_station']
    idunplug_base = datos['idunplug_base']
    idplug_base = datos['idplug_base']
    travel_time = datos['travel_time']
    Fichero = datos['Fichero']
    
    df = cursor.execute("INSERT INTO bicimad (fecha, ageRange, user_type, idunplug_station, idplug_station, idunplug_base, idplug_base, travel_time, Fichero) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", str(fecha), str(ageRange), str(user_type), str(idunplug_station), str(idplug_station), str(idunplug_base), str(idplug_base), str(travel_time),"000000")
    cnxn.commit()
    print('Rows inserted: ' + str(df.rowcount))
    print('To get the inserted row execute http://127.0.0.1:6789/api/v1.0/data/verintroducida/2022-10-03 en su navegador')
    return "Query executed succesfully."

@app.route('/api/v1.0/data/verintroducida/<fecha>',methods =['GET'])
def verNuevaIntroducida(fecha):
    # http://127.0.0.1:6789/api/v1.0/data/verintroducida/2022-10-03
    df = cursor.execute("SELECT * FROM bicimad WHERE fecha = ?", str(fecha))
    rows = df.fetchall()
    data_frame = pd.DataFrame([list(row) for row in rows],columns=["fecha", "ageRange", "user_type", "idunplug_station", "idplug_station", "idunplug_base", "idplug_base", "travel_time", "Fichero"])
    return data_frame.to_html()

@app.route('/api/v1.0/data/modificar',  methods=['PUT'])
@auth.login_required
def updateRow():
    # http://127.0.0.1:6789/api/v1.0/data/modificar
    
    #Recogemos los valores introducidos por postman en el body
    datos = request.get_json()
    #Vamos almacenando las variables para insertarlas en la query
    fecha = datos['fecha']
    ageRange = datos['ageRange']
    user_type = datos['user_type']
    idunplug_station = datos['idunplug_station']
    idplug_station = datos['idplug_station']
    idunplug_base = datos['idunplug_base']
    idplug_base = datos['idplug_base']
    travel_time = datos['travel_time']
    
    df = cursor.execute("""UPDATE bicimad SET travel_time = ? 
                        WHERE fecha = ? AND ageRange=? AND user_type=? AND idunplug_station=? 
                        AND idplug_station=? AND idunplug_base = ? AND idplug_base = ? """, str(travel_time), str(fecha),str(ageRange), str(user_type), str(idunplug_station), str(idplug_station), str(idunplug_base), str(idplug_base))
    cnxn.commit()
    print('To get the updated row execute http://127.0.0.1:6789/api/v1.0/data/verintroducida/0002-02-02')
    return "Query executed succesfully."

@app.route('/api/v1.0/data/veractualizada/<fecha>',methods =['GET'])
def verActualizada(fecha):
    # http://127.0.0.1:6789/api/v1.0/data/veractualizada/2022-10-03
    df = cursor.execute("SELECT * FROM bicimad WHERE fecha = ?", str(fecha))
    rows = df.fetchall()
    data_frame = pd.DataFrame([list(row) for row in rows],columns=["fecha", "ageRange", "user_type", "idunplug_station", "idplug_station", "idunplug_base", "idplug_base", "travel_time", "Fichero"])
    return data_frame.to_html()

@app.route('/api/v1.0/data/borrar',  methods=['DELETE'])
@auth.login_required
def delete():
    # http://127.0.0.1:6789/api/v1.0/data/borrar
    #Para ver que se ha borrado la linea usamos:
    #http://127.0.0.1:6789/api/v1.0/data/movimientos-dia/2022-10-03
    #Recogemos los valores introducidos por postman en el body
    datos = request.get_json()
    #Vamos almacenando las variables para insertarlas en la query
    #para nosotros la fecha es obligatoria
    if 'fecha' in datos:
        fecha = str(datos['fecha'])
        q = "DELETE FROM bicimad WHERE fecha = \'{}\'".format(fecha)
    if 'ageRange' in datos:
        ageRange = str(datos['ageRange'])
        q = q + ' AND ageRange = {}'.format(ageRange)
    if 'user_type' in datos:
        user_type = str(datos['user_type'])
        q = q + ' AND user_type = {}'
    if 'idunplug_station' in datos:
        idunplug_station = str(datos['idunplug_station'])
        q = q + ' AND idunplug_station = {}'.format(idunplug_station)
    if 'idplug_station' in datos:
        idplug_station = str(datos['idplug_station'])
        q = q + ' AND idplug_station = {}'.format(idplug_station)
    if 'idunplug_base' in datos:
        idunplug_base = str(datos['idunplug_base'])
        q = q + ' AND idunplug_base = {}'.format(idunplug_base)
    if 'idplug_base' in datos:
        idplug_base = str(datos['idplug_base'])
        q = q + ' AND idplug_base = {}'.format(idplug_base)
    if 'travel_time' in datos:
        travel_time = str(datos['travel_time'])
        q = q + ' AND travel_time = {}'.format(travel_time)
    if 'Fichero' in datos:
        Fichero = str(datos['Fichero'])
        q = q + ' AND Fichero = {}'.format(Fichero)
    
    print(q)
    df = cursor.execute(q)
    cnxn.commit()
    return "Query executed succesfully."

if __name__ == '__main__':
    app.run(port=6789)

ModuleNotFoundError: No module named 'flask_httpauth'